In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# 1. Setting Up the Quantum Device
dev = qml.device('default.qubit', wires=2)

# 2. Defining the QNN Ansatz
def circuit(params, wires):
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)

# 3. Defining the Cost Function
@qml.qnode(dev)
def cost_fn(params):
    circuit(params, wires=[0, 1])
    return 1 - qml.fidelity(qml.StatePrep([0, 0, 0, 1], wires=[0, 1]), wires=[0, 1])

# 4. Combined Circuit for Fidelity Measurement
@qml.qnode(dev)
def fidelity_circuit(theta_old, theta_new):
    # Apply the original parameters
    circuit(theta_old, wires=[0, 1])
    # Apply the adjoint (inverse) of the new parameters
    qml.RY(-theta_new[3], wires=1)
    qml.RY(-theta_new[2], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(-theta_new[1], wires=1)
    qml.RY(-theta_new[0], wires=0)
    return qml.probs(wires=[0, 1])

# 5. Fidelity Calculation
def compute_fidelity(theta_old, theta_new):
    probs = fidelity_circuit(theta_old, theta_new)
    fidelity = probs[0]  # Probability of |00>
    return fidelity

# 6. Custom Optimizer
def custom_train(initial_params, learning_rate=0.1, fidelity_threshold=0.99, max_iterations=100):
    params = initial_params.copy()
    cost_history = []
    fidelity_history = []
    
    for it in range(max_iterations):
        current_cost = cost_fn(params)
        cost_history.append(current_cost)
        
        # Compute gradients
        grads = qml.grad(cost_fn)(params)
        
        # Propose new parameters
        params_new = params - learning_rate * grads
        
        # Compute fidelity
        fidelity = compute_fidelity(params, params_new)
        fidelity_history.append(fidelity)
        
        # Decide to accept or reject
        if fidelity >= fidelity_threshold:
            params = params_new
            print(f"Iteration {it+1}: Cost = {current_cost:.6f}, Fidelity = {fidelity:.6f} -> Update Accepted")
        else:
            # Reduce learning rate and do not update
            learning_rate *= 0.5
            print(f"Iteration {it+1}: Cost = {current_cost:.6f}, Fidelity = {fidelity:.6f} -> Update Rejected, New Learning Rate = {learning_rate}")
        
        # Early stopping
        if current_cost < 1e-6:
            print("Convergence achieved.")
            break
    
    return params, cost_history, fidelity_history

# 7. Built-In Adam Optimizer Training
def adam_train(initial_params, max_iterations=100):
    opt = qml.AdamOptimizer(stepsize=0.1)
    params = initial_params.copy()
    cost_history = []
    
    for it in range(max_iterations):
        current_cost = cost_fn(params)
        cost_history.append(current_cost)
        
        # Update parameters
        params, _ = opt.step_and_cost(cost_fn, params)
        
        print(f"Iteration {it+1}: Cost = {current_cost:.6f}")
        
        # Early stopping
        if current_cost < 1e-6:
            print("Convergence achieved.")
            break
    
    return params, cost_history

# 8. Running the Training
# Set random seed
np.random.seed(42)

# Initialize parameters
initial_params_custom = np.random.uniform(0, 2*np.pi, 4)
initial_params_adam = initial_params_custom.copy()

# Train using custom optimizer
print("Training with Custom Fidelity-Based Optimizer:")
final_params_custom, cost_history_custom, fidelity_history_custom = custom_train(
    initial_params=initial_params_custom,
    learning_rate=0.1,
    fidelity_threshold=0.99,
    max_iterations=50
)

print("\nTraining with Adam Optimizer:")
# Train using Adam optimizer
final_params_adam, cost_history_adam = adam_train(
    initial_params=initial_params_adam,
    max_iterations=50
)

# 9. Plotting the Results
plt.figure(figsize=(12, 5))

# Plot Cost Function
plt.subplot(1, 2, 1)
plt.plot(cost_history_custom, label='Custom Optimizer')
plt.plot(cost_history_adam, label='Adam Optimizer')
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Cost Function Convergence')
plt.yscale('log')
plt.legend()

# Plot Fidelity
plt.subplot(1, 2, 2)
plt.plot(fidelity_history_custom, label='Fidelity')
plt.xlabel('Iteration')
plt.ylabel('Fidelity')
plt.title('Fidelity Between Parameter Updates')
plt.legend()

plt.tight_layout()
plt.show()


Training with Custom Fidelity-Based Optimizer:


AttributeError: Module pennylane has no attribute fidelity